In [2]:
%matplotlib inline
import random
random.seed(1234)

import pandas as pd
import gzip
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# import nltk
# nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
import spacy
import matplotlib.pyplot as plt
import pyLDAvis #python library for interactive topic model visualization
# import pyLDAvis.gensim
pyLDAvis.enable_notebook()

# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

# import warnings
# warnings.filterwarnings("ignore",category=DeprecationWarning)

import pickle
import numpy as np

# read data 

In [5]:
# def parse(path):
#   g = gzip.open(path, 'rb')
#   for l in g:
#     yield eval(l)

# def getDF(path):
#   i = 0
#   df = {}
#   for d in parse(path):
#     df[i] = d
#     i += 1
#   return pd.DataFrame.from_dict(df, orient='index')

# df = getDF('./reviews_Pet_Supplies.json.gz') #1,235,316
# metadata_df = getDF('./meta_Pet_Supplies.json.gz')
# merged_df = pd.merge(df, metadata_df, on = 'asin', how = 'left')

(1235316, 17)

In [ ]:
# columns = ['category_' + str(i) for i in range(1, 6)] 

In [ ]:
# total_l = merged_df.categories.values.tolist()

In [ ]:
# len(total_l[152][0])
# leng = [len(l[0]) for l in total_l]

In [ ]:
# merged_df2 = pd.DataFrame([l[0] for l in total_l], columns = columns)

In [ ]:
# return_df = pd.concat([merged_df, merged_df2],axis = 1)

In [ ]:
#return_df.to_csv('all_df.csv')

In [ ]:
# merged_df = pd.read_csv('./all_df.csv', index_col = False)

In [8]:
# from tqdm.notebook import tqdm as tqdm
# tqdm.pandas()

In [ ]:
#merged_df.loc[(merged_df.category_2 == 'Dogs') | (merged_df.category_2 == 'Cats')].reset_index(drop = True).to_csv('dogs_cats_data.csv', index = False)

In [3]:
df = pd.read_csv('dogs_cats_data.csv', index_col = False)

In [4]:
vc = df.loc[df.category_2 == 'Dogs', ['category_2', 'category_3', 'category_4']].value_counts()

In [5]:
vc[vc > 1000]

category_2  category_3                    category_4                
Dogs        Treats                        Cookies, Biscuits & Snacks    10843
            Toys                          Squeak Toys                    7220
            Food                          Dry                            5872
            Toys                          Chew Toys                      5459
            Health Supplies               Supplements & Vitamins         5097
            Toys                          Balls                          4454
            Collars, Harnesses & Leashes  Collars                        3264
                                          Harnesses                      2619
            Feeding & Watering Supplies   Fountains                      2346
            Health Supplies               Flea, Lice & Tick Control      2183
            Collars, Harnesses & Leashes  Leashes                        2010
            Health Supplies               Dental Care                    

In [8]:
vc[(vc > 500) & (vc <= 1000)]

category_2  category_3                   category_4                     
Dogs        Training & Behavior Aids     Repellents                         892
            Treats                       Bones                              866
            Toys                         Ropes                              813
            Health Supplies              Ear Care                           768
            Grooming                     Electric Clippers & Blades         703
                                         Brushes                            693
                                         Shedding Tools                     681
            Feeding & Watering Supplies  Travel Bowls                       638
            Health Supplies              Relaxants                          597
            Feeding & Watering Supplies  Automatic Feeders                  537
            Training & Behavior Aids     Training Collars                   534
            Health Supplies              Eye Ca

In [9]:
vc[(vc <= 500) & (vc > 300)]

category_2  category_3             category_4             
Dogs        Toys                   Flying Discs               425
            Apparel & Accessories  Hoodies                    382
            Beds & Furniture       Bed Mats                   381
            Doors, Gates & Ramps   Doors                      342
            Beds & Furniture       Stairs & Steps             339
            Doors, Gates & Ramps   Playpens                   339
            Grooming               Scissors                   337
            Food                   Dehydrated/Freeze-Dried    305
dtype: int64

In [10]:
merged_df1 = pd.DataFrame(vc[vc <= 300]).assign(updated_category_4 = 'Others')

In [11]:
vc2 = df.loc[df.category_2 == 'Cats', ['category_2', 'category_3', 'category_4']].value_counts()

In [12]:
vc2[vc2>1000]

category_2  category_3              category_4            
Cats        Food                    Wet                       3955
            Beds & Furniture        Beds & Sofas              2366
            Toys                    Catnip Toys               2268
            Litter & Housebreaking  Litter Boxes              2257
            Toys                    Mice & Animal Toys        2091
            Food                    Dry                       1961
            Treats                  Snacks                    1578
            Health Supplies         Relaxants                 1432
            Litter & Housebreaking  Litter                    1242
            Beds & Furniture        Activity Trees            1213
            Toys                    Feather Toys              1140
            Beds & Furniture        Scratching Pads           1075
            Health Supplies         Supplements & Vitamins    1072
dtype: int64

In [13]:
vc2[(vc2 > 500) & (vc2 <= 1000)]

category_2  category_3                   category_4               
Cats        Health Supplies              Flea, Lice & Tick Control    708
            Toys                         Balls                        646
            Beds & Furniture             Scratching Posts             582
            Treats                       Catnip                       578
            Litter & Housebreaking       Litter Scoops                532
            Feeding & Watering Supplies  Fountains                    527
            Beds & Furniture             Cat Houses & Condos          515
            Toys                         Laser Toys                   511
dtype: int64

In [14]:
merged_df2 = pd.DataFrame(vc2[vc2 <= 300]).assign(updated_category_4 = 'Others')

In [15]:
updated_df = pd.merge(df, pd.concat([merged_df1, merged_df2]), on = ['category_2', 'category_3', 'category_4'], how = 'left')

In [17]:
updated_df['final_category_4'] = np.where(updated_df['updated_category_4'].isnull(), updated_df['category_4'], updated_df['updated_category_4'])

In [18]:
updated_df.loc[(updated_df.category_2 == 'Dogs') & (updated_df.final_category_4 == 'Others'), ['category_2', 'category_3', 'final_category_4']].value_counts()

category_2  category_3                    final_category_4
Dogs        Apparel & Accessories         Others              1801
            Grooming                      Others              1482
            Health Supplies               Others              1048
            Training & Behavior Aids      Others               768
            Litter & Housebreaking        Others               314
            Beds & Furniture              Others               299
            Collars, Harnesses & Leashes  Others               271
            Carriers & Travel Products    Others               226
            Doors, Gates & Ramps          Others               177
            Feeding & Watering Supplies   Others               157
            Treats                        Others               145
            Houses, Kennels & Pens        Others               115
            Food                          Others                48
            Toys                          Others                36
dty

In [19]:
updated_df.loc[(updated_df.category_2 == 'Cats') & (updated_df.final_category_4 == 'Others'), ['category_2', 'category_3', 'final_category_4']].value_counts()

category_2  category_3                     final_category_4
Cats        Health Supplies                Others              630
            Feeding & Watering Supplies    Others              566
            Grooming                       Others              450
            Litter & Housebreaking         Others              277
            Cat Flaps, Steps, Nets & Pens  Others              217
            Collars, Harnesses & Leashes   Others              193
            Educational Repellents         Others              192
            Carriers & Strollers           Others               88
            Beds & Furniture               Others               83
dtype: int64

In [30]:
updated_df2 = updated_df.drop(columns = [0])

In [31]:
updated_df2.to_csv('recategorized_dogs_cats_data.csv', index = False)